## Bird species classification

**Tout d'abord, on importe les modules**

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F 
import matplotlib.pyplot as plt
import torchvision.transforms as T #Pour changer la taille des images, convertir les images PIl en tenseur Pytorch
import torchvision.models as models #Pour récupérer les réseaux pré-entrainés
from torch.utils.data import DataLoader #Pour segmenter notre dataset en batch
from torchvision.utils import make_grid #Pour afficher quelques images du dataset

**On prépare les données**

In [ ]:
train_path = r"C:/Users/thete/OneDrive/Documents/ProjetS3/train"
valid_path = r"C:/Users/thete/OneDrive/Documents/ProjetS3/valid"
test_path = r"C:/Users/thete/OneDrive/Documents/ProjetS3/valid"

In [ ]:
transform_dataset = T.Compose([
    T.Resize((128,128)),
    T.RandomHorizontalFlip(),
    T.ToTensor()
])

train_dataset = torchvision.datasets.ImageFolder(root=train_path , transform=transform_dataset)
valid_dataset = torchvision.datasets.ImageFolder(root=valid_path , transform=transform_dataset)
test_dataset = torchvision.datasets.ImageFolder(root=test_path , transform=transform_dataset)

In [ ]:
number_of_species = len(train_dataset.classes)
print("Les oiseaux seront classifiés parmis" , number_of_species , "espèces d'oiseaux.")

In [ ]:
batch_size = 32 #doit être une puissance de 2

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=False)


**Affichage de certaines images de notre dataset**

In [ ]:
def show_images(train_dl):
    for images, labels in train_dl:
        fig, ax = plt.subplots(figsize=(10,10))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images[:32], nrow=8).permute(1,2,0))
        break

In [ ]:
bird_images = show_images(train_dataloader)
bird_images

**Allouer les calculs et les données au GPU**

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")
    
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dataloader, device):
        self.dataloader = dataloader
        self.device = device
        
    def __iter__(self):
        for x in self.dataloader:
            yield to_device(x, self.device)
            
    def __len__(self):
        return len(self.dataloader)

In [ ]:
device = get_device()
device

In [ ]:
train_dataloader = DeviceDataLoader(train_dataloader, device)
valid_dataloader = DeviceDataLoader(valid_dataloader, device)

**Definition du modèle**

In [ ]:
#Fonction calculant la précision, la métrique choisie par l'équipe, elle sera utilisée dans l'entraînement pour calculer la précision à chaque epoch

def accuracy(out, labels):
    _, preds = torch.max(out, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {"valid_loss": loss.detach(), "valid_acc": acc}
    
    def validation_epoch_end(self, outputs):
        batch_loss = [x["valid_loss"] for x in outputs]
        epoch_loss = torch.stack(batch_loss).mean()
        batch_acc = [x["valid_acc"] for x in outputs]
        epoch_acc = torch.stack(batch_acc).mean()
        return {"valid_loss": epoch_loss.item(), "valid_acc": epoch_acc.item()}
    
    def epoch_end(self, epoch, epochs, result):
        print("Epoch: [{}/{}], last_lr: {:.6f}, train_loss: {:.4f}, valid_loss: {:.4f}, valid_acc: {:.4f}".format(
            epoch+1, epochs, result["lrs"][-1], result["train_loss"], result["valid_loss"], result["valid_acc"]))

In [ ]:
class model(ImageClassificationBase):
    def __init__(self, num_classes):
        super().__init__()
        self.network = models.resnet18(pretrained=True)
        number_of_features = self.network.fc.in_features
        self.network.fc = nn.Linear(number_of_features, number_of_species)
        
    def forward(self, xb):
        return self.network(xb)
    
    def freeze(self):
        for param in self.network.parameters():
            param.requires_grad= False
        for param in self.network.fc.parameters():
            param.requires_grad= True
        
    def unfreeze(self):
        for param in self.network.parameters():
            param.requires_grad= True

In [ ]:
model = to_device(model(num_classes=number_of_species), device)
model

In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
                           nn.BatchNorm2d(out_channels),
                          nn.ReLU())]
    
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)
    
    
class resnet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128,128))
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(8), nn.Flatten(), nn.Linear(512, num_classes))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [27]:
@torch.no_grad()
def evaluate(model, valid_dataloader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in valid_dataloader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]
    
def fit_one_cycle(epochs, max_lr, model, train_dataloader, valid_dataloader, weight_decay=0, opt_func=torch.optim.Adam):
    torch.cuda.empty_cache()
    
    history = []
    
    opt = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr, epochs=epochs,
                                               steps_per_epoch=len(train_dataloader))
    
    for epoch in range(epochs):
        model.train()
        train_loss = []
        lrs = []
        for batch in train_dataloader:
            loss = model.training_step(batch)
            train_loss.append(loss)
            loss.backward()
                
            opt.step()
            opt.zero_grad()
            
            lrs.append(get_lr(opt))
            sched.step()
            
        result = evaluate(model, valid_dataloader)
        result["train_loss"] = torch.stack(train_loss).mean().item()
        result["lrs"] = lrs
        model.epoch_end(epoch, epochs, result)
        history.append(result)
    return history

In [28]:
result = [evaluate(model, valid_dataloader)]
result

[{'valid_loss': 2.2327828407287598, 'valid_acc': 0.25}]

In [29]:
model.freeze()

In [35]:
epochs = 30
max_lr = 10e-5
grad_clip = 0.1
weight_decay = 10e-4
opt_func = torch.optim.Adam

In [36]:
%%time

history = fit_one_cycle(epochs, max_lr, model, train_dataloader, valid_dataloader, weight_decay=weight_decay, opt_func=opt_func)

Epoch: [1/30], last_lr: 0.000006, train_loss: 1.1712, valid_loss: 1.1801, valid_acc: 0.5083
Epoch: [2/30], last_lr: 0.000014, train_loss: 1.1711, valid_loss: 1.1850, valid_acc: 0.5083
Epoch: [3/30], last_lr: 0.000027, train_loss: 1.1522, valid_loss: 1.2145, valid_acc: 0.4667
Epoch: [4/30], last_lr: 0.000043, train_loss: 1.1611, valid_loss: 1.1846, valid_acc: 0.5167
Epoch: [5/30], last_lr: 0.000060, train_loss: 1.1451, valid_loss: 1.1416, valid_acc: 0.5417
Epoch: [6/30], last_lr: 0.000075, train_loss: 1.1343, valid_loss: 1.1140, valid_acc: 0.5417
Epoch: [7/30], last_lr: 0.000088, train_loss: 1.1078, valid_loss: 1.1465, valid_acc: 0.4750
Epoch: [8/30], last_lr: 0.000097, train_loss: 1.0876, valid_loss: 1.1090, valid_acc: 0.5500
Epoch: [9/30], last_lr: 0.000100, train_loss: 1.0598, valid_loss: 1.0494, valid_acc: 0.6083
Epoch: [10/30], last_lr: 0.000099, train_loss: 1.0494, valid_loss: 1.0135, valid_acc: 0.5833
Epoch: [11/30], last_lr: 0.000098, train_loss: 1.0131, valid_loss: 1.0532, vali

In [ ]:
model.unfreeze()

In [ ]:
epochs = 5
max_lr = 10e-5
grad_clip = 0.1
weight_decay = 10e-4
opt_func = torch.optim.Adam

In [37]:
%%time

history += fit_one_cycle(epochs, max_lr, model, train_dataloader, valid_dataloader, weight_decay=weight_decay, opt_func=opt_func)


Epoch: [1/30], last_lr: 0.000006, train_loss: 0.8807, valid_loss: 0.9172, valid_acc: 0.6500
Epoch: [2/30], last_lr: 0.000014, train_loss: 0.8847, valid_loss: 0.9109, valid_acc: 0.6583
Epoch: [3/30], last_lr: 0.000027, train_loss: 0.8939, valid_loss: 0.9100, valid_acc: 0.6500
Epoch: [4/30], last_lr: 0.000043, train_loss: 0.8746, valid_loss: 0.9171, valid_acc: 0.6500
Epoch: [5/30], last_lr: 0.000060, train_loss: 0.8925, valid_loss: 0.9253, valid_acc: 0.6167
Epoch: [6/30], last_lr: 0.000075, train_loss: 0.8651, valid_loss: 0.8818, valid_acc: 0.6417
Epoch: [7/30], last_lr: 0.000088, train_loss: 0.8682, valid_loss: 0.8785, valid_acc: 0.6833
Epoch: [8/30], last_lr: 0.000097, train_loss: 0.8431, valid_loss: 0.8655, valid_acc: 0.6917
Epoch: [9/30], last_lr: 0.000100, train_loss: 0.8331, valid_loss: 0.8633, valid_acc: 0.6917
Epoch: [10/30], last_lr: 0.000099, train_loss: 0.8291, valid_loss: 0.8439, valid_acc: 0.6583
Epoch: [11/30], last_lr: 0.000098, train_loss: 0.8139, valid_loss: 0.8332, vali

961294e-05,
   2.53326288150591e-05,
   2.6920068893634456e-05]},
 {'valid_loss': 1.184573769569397,
  'valid_acc': 0.5166666507720947,
  'train_loss': 1.161134123802185,
  'lrs': [2.8546180408146166e-05,
   3.020845601250176e-05,
   3.190433259820345e-05,
   3.363119524647569e-05,
   3.5386381260280335e-05,
   3.716718427000253e-05,
   3.897085840647644e-05,
   4.079462253491654e-05,
   4.263566454322584e-05]},
 {'valid_loss': 1.1416089534759521,
  'valid_acc': 0.5416666865348816,
  'train_loss': 1.145128846168518,
  'lrs': [4.449114567806892e-05,
   4.635820492202379e-05,
   4.823396340506344e-05,
   5.011552884356471e-05,
   5.2000000000000004e-05,
   5.3884471156435286e-05,
   5.5766036594936555e-05,
   5.764179507797621e-05,
   5.95088543219311e-05]},
 {'valid_loss': 1.1140276193618774,
  'valid_acc': 0.5416666865348816,
  'train_loss': 1.1342936754226685,
  'lrs': [6.136433545677415e-05,
   6.320537746508346e-05,
   6.502914159352356e-05,
   6.683281572999748e-05,
   6.8613618739

In [42]:
print("Au cours de l'entrainement, la précision maximale de notre modèle est de" ,round(max([history[i]["valid_acc"] for i in range(len(history))]) , 3))

Au cours de l'entrainement, la précision maximale de notre modèle est de 0.742


**Predictions sur le jeu de donnée de test**

In [ ]:
def prediction(model, images):
    xb = to_device(images.unsqueeze(0), device)
    out = model(xb)
    _, preds = torch.max(out, dim=1)
    predictions = test_ds.classes[preds[0].item()]
    return predictions

In [ ]:
#Exemple of a prediction
indice = 42

images, labels = test_ds[indice]
print("Label: ", test_ds.classes[labels])
print("Prediction: ", prediction(model, images))
plt.imshow(images.permute(1,2,0))